# Match Naturalis Collectors to Wikidata Items Using *Nearest Neighbour*, `eventDate` Involved

In this example we add `eventDate` of the source data, when the sample/occurrence was collected, to have a time reference, when the collector should have been  alive. 

Basically we …

- match of `canonical_string` of WikiData to `canonical_string` of the source collectors (abbreviated names and full names, if given), and
- parse collector source names beforehand to get individual names out of name lists in the source data, we have used <https://libraries.io/rubygems/dwc_agent>, and in general we
- follow the example of Niels Klazenga <https://github.com/nielsklazenga/avh-collectors/blob/master/match_names_to_wikidata_items.ipynb>
- calculate the name distance and also a similarity score for the matching of life span with the (sampling) `eventDate`, and finally we
- write the output to provide a DarwinCore attribution structure (for `verbatimName` we would need the `source_data` name(s))

Technical Notes — Review Code perhaps:

- TODO review score calculation of the matching of relating eventData with range of yob, yod
- TODO review DwC agent output, keep at this time custom columns for filter-sort-evaluation convenience
- (NN ⇌ Cosine) refactor relation: wd_matchtest ⇌ wikidata_unique (replaced wd_matchtest → wikidata_unique)

### Load Wikidata Data Set

Use Jupyter Notebook [create_wikidata_datasets_botanists-altlabel.ipynb](./create_wikidata_datasets_botanists-altlabel.ipynb) to generate matching data of botanists.

Now load the data and make them unique …

In [1]:
import pandas as pd
import pprint, time, os

explain_and_show_the_data = True
this_timestamp_for_data=20260210
# this_timestamp_for_data=time.strftime('%Y%m%d')

wikidata = pd.read_csv(
    # "data/wikidata_persons_botanists_20231030_1539.csv", # inverse match: [particle +] family, given
    # "data/wikidata_persons_botanists_20231116.csv",        # match: given [+ particle] + family[+ , suffix]
    # "data/wikidata_persons_botanists_20240312.csv", # with itemLabel + altLabel wyb, wye removed
    "data/wikidata_persons_botanists_20260210.csv",
    index_col=0, low_memory=False,
    dtype={
        'yob':'Int32',
        'yod':'Int32',
        'wyb':'Int32',
        'wye':'Int32'
    }    
)
# # # # # # # # # # # # # # # # 
# def convert_to_datetime(year):
#     year = int(year)
#     days = int((year - int(year)) * 365.25)
#     base_date = pd.to_datetime(f'{year}-01-01', errors="ignore")
#     # ,format='%Y-%m-%d', errors='coerce'
#     # base_date = pd.Period(year, freq='D')
#     return base_date + pd.DateOffset(days=days)
# 
# def convert_to_datetime(year):
#     base_date = pd.to_datetime(f'{year}-01-01', errors="ignore")
# print(wikidata.dtypes)

# wikidata['yob_converted'] = wikidata['yob'].apply(convert_to_datetime)
# wikidata['yod_converted'] = wikidata['yod'].apply(lambda x: pd.Period(x, freq='Y')) # Given date string "-286" not likely a datetime
# # # # # # # # # # # # # # # # 

if explain_and_show_the_data:
    pprint.pprint(wikidata.columns)
    display(wikidata.head())

Index(['item', 'itemLabel', 'surname', 'initials', 'canonical_string',
       'canonical_string_fullname', 'orcid', 'viaf', 'isni', 'harv', 'ipni',
       'abbr', 'bionomia_id', 'yob', 'yod', 'wikidata_link', 'orcid_link',
       'harv_link', 'ipni_link', 'bionomia_link'],
      dtype='str')


,item,itemLabel,surname,initials,canonical_string,canonical_string_fullname,orcid,viaf,isni,harv,ipni,abbr,bionomia_id,yob,yod,wikidata_link,orcid_link,harv_link,ipni_link,bionomia_link
0,http://www.wikidata.org/entity/Q100142069,Frida Eggens,NaN,NaN,Eggens,Eggens,NaN,NaN,NaN,NaN,20045232-1,Eggens,NaN,<NA>,<NA>,http://www.wikidata.org/wiki/Q100142069,NaN,NaN,https://www.ipni.org/a/20045232-1,NaN
1,http://www.wikidata.org/entity/Q100142069,Frida Eggens,Frida,F.,F. Eggens,Frida Eggens,NaN,NaN,NaN,NaN,20045232-1,Eggens,NaN,<NA>,<NA>,http://www.wikidata.org/wiki/Q100142069,NaN,NaN,https://www.ipni.org/a/20045232-1,NaN
2,http://www.wikidata.org/entity/Q100146795,Elizabeth Harrison,Elizabeth,E.,E. Harrison,Elizabeth Harrison,NaN,NaN,NaN,NaN,NaN,NaN,Q100146795,1792,1834,http://www.wikidata.org/wiki/Q100146795,NaN,NaN,NaN,https://bionomia.net/Q100146795
3,http://www.wikidata.org/entity/Q100146795,Elizabeth Harrison,NaN,NaN,Mrs A. H.,Mrs A. H.,NaN,NaN,NaN,NaN,NaN,NaN,Q100146795,1792,1834,http://www.wikidata.org/wiki/Q100146795,NaN,NaN,NaN,https://bionomia.net/Q100146795
4,http://www.wikidata.org/entity/Q100146795,Elizabeth Harrison,Mrs Arnold,M. A.,M. A. Harrison,Mrs Arnold Harrison,NaN,NaN,NaN,NaN,NaN,NaN,Q100146795,1792,1834,http://www.wikidata.org/wiki/Q100146795,NaN,NaN,NaN,https://bionomia.net/Q100146795


In [2]:
# compile data having only unique canonical strings
# group by canonical name/string, count douplicated names
wd_matchtest = wikidata.groupby('canonical_string').agg({'item': ['count']}).reset_index()
wd_matchtest_fullnames = wikidata.groupby('canonical_string_fullname').agg({'item': ['count']}).reset_index()

if explain_and_show_the_data:
    display(wd_matchtest)
    display(wd_matchtest_fullnames)

,canonical_string,item
,,count
0,"""F."" Ryser",1
1,"""N.A. Antipova"" (lapsus)",1
2,"""N.A.Antipova"" (lapsus)",1
3,"""The grandmother of female scientists in Ghana""",1
4,"""Н. А. Антипова"" (lapsus)",1
...,...,...
171443,赵云鹏,1
171444,郭亚龙,1
171445,金井弘夫(Hiroo Kanai),1


,canonical_string_fullname,item
,,count
0,"""Fritz"" Ryser",1
1,"""N.A. Antipova"" (lapsus)",1
2,"""N.A.Antipova"" (lapsus)",1
3,"""The grandmother of female scientists in Ghana""",1
4,"""Н. А. Антипова"" (lapsus)",1
...,...,...
204788,赵云鹏,1
204789,郭亚龙,1
204790,金井弘夫(Hiroo Kanai),1


### Load Collectors Data Set

Data sources:

- Jupyter Notebook for [create_naturalis_gbif-occurrence_collectors_eventDate_dataset.ipynb](./create_naturalis_gbif-occurrence_collectors_eventDate_dataset.ipynb)

Then parse collector names to get single, separate collector names using `dwcagent`, use ruby gem package available at  <https://rubygems.org/gems/dwc_agent>:

- see [./bin/README.md](bin/README.md) to use ruby script `./bin/agent_parse4tsv.rb` for parsing text lines like `"Abbe,L.B., Abbe,E.C., Smitinand,T. & Rollet,B."`


In [3]:
# atomized names were parsed already by ruby gem package: dwcagent —
# they can contain also the same name accross multiple rows — 
# it’s probably better for the matching to make the name rows unique later on

# collectors = pd.read_csv("data/naturalis_collectors_20230510_1429_single-line_parsed_unique_names.tab", sep="\t")
collectors = pd.read_csv("./data/Naturalis_doi-10.15468-dl.uw8rxk/occurrence_recordedBy_eventDate_occurrenceIDs_20230913_parsed.tsv", sep="\t", low_memory=False)

collectors.dropna(subset=['family'], inplace=True) # remove where family was NA, e.g. from originally «??» aso.

print("modify time using pd.Periode(…) to get it work also on very old dates...")
for col in ['eventDate_mean', 'eventDate_min', 'eventDate_max']:
    print("- convert", col, "to pd.Period(...) in collectors")
    collectors[col] = collectors[col].apply(
        lambda x: pd.Period(
            x, freq='s' if col.lower().endswith('mean') else 'D' # Seconds or Day level
        )
    ) # D=day level
    # see https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#timeseries-period-aliases
print("modifing done.")

collectors.sort_values(by=['family', 'given','occurrenceID_firstsample'], inplace=True)
if explain_and_show_the_data: display(collectors)

modify time using pd.Periode(…) to get it work also on very old dates...
- convert eventDate_mean to pd.Period(...) in collectors
- convert eventDate_min to pd.Period(...) in collectors
- convert eventDate_max to pd.Period(...) in collectors
modifing done.


,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
28,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A.,parsed:A,cleaned:A,6,https://data.biodiversitydata.nl/naturalis/spe...,1962-09-11 09:36:00,1960-04-01,1963-04-27
86139,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Kampen PN van; A,parsed:Kampen PN van<SEP>A,cleaned:<SEP>A,1,https://data.biodiversitydata.nl/naturalis/spe...,1899-08-07 00:00:00,1899-08-07,1899-08-07
180483,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Wendt T; Villalobos C A; A; Navarrete I,parsed:T. Wendt<SEP>A. Villalobos C<SEP>A<SEP>...,cleaned:T. Wendt<SEP>A. Villalobos C<SEP>A<SEP...,5,https://data.biodiversitydata.nl/naturalis/spe...,1981-12-26 00:00:00,1981-03-20,1983-05-18
172221,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown; A,parsed:A,cleaned:A,1,https://data.biodiversitydata.nl/naturalis/spe...,NaT,NaT,NaT
55251,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Fuentes C; A; Rosa de la,parsed:C. Fuentes<SEP>A<SEP>Rosa de la,cleaned:C. Fuentes<SEP>A<SEP>,1,https://data.biodiversitydata.nl/naturalis/spe...,1997-02-01 00:00:00,1997-02-01,1997-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
189370,Štepánek,J.,NaN,NaN,NaN,NaN,NaN,NaN,Štepánek J; Trávnícek B,parsed:J. Štepánek<SEP>B. Trávnícek,cleaned:J. Štepánek<SEP>B. Trávnícek,2,https://data.biodiversitydata.nl/naturalis/spe...,1998-11-06 12:00:00,1992-05-15,2005-04-30
89855,Štepánek,J.,NaN,NaN,NaN,NaN,NaN,NaN,Kirschnerová L; Kirschner J; Štepánek J,parsed:L. Kirschnerová<SEP>J. Kirschner<SEP>J....,cleaned:L. Kirschnerová<SEP>J. Kirschner<SEP>J...,1,https://data.biodiversitydata.nl/naturalis/spe...,1983-05-09 00:00:00,1983-05-09,1983-05-09
189355,Štepánek,J.,NaN,NaN,NaN,NaN,NaN,NaN,Štepánek J; Jakl J,parsed:J. Štepánek<SEP>J. Jakl,cleaned:J. Štepánek<SEP>J. Jakl,1,https://data.biodiversitydata.nl/naturalis/spe...,2003-06-01 00:00:00,2003-06-01,2003-06-01
65698,Štepánek,J.,NaN,NaN,NaN,NaN,NaN,NaN,Hadinec J; Kirschner J; Sourkova M; Štepánek J,parsed:J. Hadinec<SEP>J. Kirschner<SEP>M. Sour...,cleaned:J. Hadinec<SEP>J. Kirschner<SEP>M. Sou...,1,https://data.biodiversitydata.nl/naturalis/spe...,1978-05-17 00:00:00,1978-05-17,1978-05-17


#### Check Composition of Parsed Collector Data

In [4]:
# TODO review code of abbreviated names and full name matching
if explain_and_show_the_data: 
    criterion_fullnames = collectors.given.str.contains('^\\w{3,}', na=False)
    print("Show collecors given name has (propably) a full name (%s records) …" % len(collectors[criterion_fullnames].index))
    display(collectors[criterion_fullnames])

Show collecors given name has (propably) a full name (6111 records) …


,family,given,suffix,particle,dropping_particle,nick,appellation,title,source_data,parsed_names,cleaned_names,occurrenceID_count,occurrenceID_firstsample,eventDate_mean,eventDate_min,eventDate_max
141198,A-M-V-J,Renier,NaN,NaN,NaN,NaN,NaN,NaN,Renier A-M-V-J,parsed:Renier A-M-V-J,cleaned:Renier A-M-V-J,1,https://data.biodiversitydata.nl/naturalis/spe...,NaT,NaT,NaT
75657,A-ts'ai,Hsieh,NaN,NaN,NaN,NaN,NaN,NaN,Hsieh A-ts'ai,parsed:Hsieh A-ts'ai,cleaned:Hsieh A-ts'ai,1,https://data.biodiversitydata.nl/naturalis/spe...,1929-05-21 00:00:00,1929-05-21,1929-05-21
162955,A. Kneucker T,Stuckert,NaN,in,NaN,NaN,NaN,NaN,Stuckert in A. Kneucker T,parsed:Stuckert in A. Kneucker T,cleaned:Stuckert in A. Kneucker T,4,https://data.biodiversitydata.nl/naturalis/spe...,1902-01-01 00:00:00,1902-01-01,1902-01-01
83431,AFle,Jolis,NaN,NaN,NaN,NaN,NaN,NaN,Jolis AFle,parsed:Jolis AFle,cleaned:Jolis AFle,420,https://data.biodiversitydata.nl/naturalis/spe...,1860-07-06 19:47:47,1800-01-01,1983-10-04
14090,Aaaa,Bellynck,NaN,NaN,NaN,NaN,NaN,NaN,Bellynck AAAA,parsed:Bellynck AAAA,cleaned:Bellynck Aaaa,6,https://data.biodiversitydata.nl/naturalis/spe...,NaT,NaT,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
144657,le,Roux A.,NaN,NaN,NaN,NaN,NaN,NaN,Roux A le; Lloyd JW,parsed:Roux A. le<SEP>J.W. Lloyd,cleaned:Roux A. le<SEP>J.W. Lloyd,1,https://data.biodiversitydata.nl/naturalis/spe...,1985-03-09 00:00:00,1985-03-09,1985-03-09
144659,le,Roux A.,NaN,NaN,NaN,NaN,NaN,NaN,Roux A le; Ramsey M,parsed:Roux A. le<SEP>M. Ramsey,cleaned:Roux A. le<SEP>M. Ramsey,9,https://data.biodiversitydata.nl/naturalis/spe...,1977-12-16 21:20:00,1972-09-13,1978-09-13
163375,le,Sueur F.A.,NaN,NaN,NaN,NaN,NaN,NaN,Sueur FA le,parsed:Sueur F.A. le,cleaned:Sueur F.A. le,23,https://data.biodiversitydata.nl/naturalis/spe...,1977-01-09 19:12:00,1951-03-15,1981-04-01
167408,le,Testu G.M.P.C.,NaN,NaN,NaN,NaN,NaN,NaN,Testu GMPC le,parsed:Testu G.M.P.C. le,cleaned:Testu G.M.P.C. le,3,https://data.biodiversitydata.nl/naturalis/spe...,NaT,NaT,NaT


In [5]:
# check the name-parsed columns if they are empty or need to be considerd as data for matching or not
if explain_and_show_the_data: 
    for parsed_name_part in ["particle", "suffix", "dropping_particle", "appellation"]:
        test_collectors = collectors.loc[(collectors[parsed_name_part].isna() == False)]
        print("\n----------------------------------------\nshow names with **%s** found %s records:\n" % (parsed_name_part, len(test_collectors.index)))
        display(test_collectors.head().get(["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title"]))    


----------------------------------------
show names with **particle** found 4782 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
162955,A. Kneucker T,Stuckert,NaN,in,NaN,NaN,NaN,NaN
68,Aa,H. A. van der,NaN,van,NaN,NaN,NaN,NaN
52,Aa,H.A.,NaN,van der,NaN,NaN,NaN,NaN
81,Aalst,Mdjm,NaN,van,NaN,NaN,NaN,NaN
89,Aanen,D.K.,NaN,van der,NaN,NaN,NaN,NaN



----------------------------------------
show names with **suffix** found 22 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
188603,Bakker,Zinderen,Sr.,NaN,NaN,NaN,NaN,NaN
62158,Gradstein,NaN,SR,van,NaN,NaN,NaN,NaN
62137,Gradstein,NaN,SR,van,NaN,NaN,NaN,NaN
89502,Leopold,King,III,NaN,NaN,NaN,NaN,NaN
159272,Maurit,Flora,II,NaN,NaN,NaN,NaN,NaN



----------------------------------------
show names with **dropping_particle** found 0 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title



----------------------------------------
show names with **appellation** found 1 records:



,family,given,suffix,particle,dropping_particle,nick,appellation,title
19154,McCullogh,NaN,NaN,NaN,NaN,NaN,Mrs,NaN


Compile and compose `canonical_string…` of the collector data that we will later match the WikiData names with:

In [6]:
if explain_and_show_the_data: print("combine parts of names similar to WikiData's given name labels")

# 1. Prepare canonical name with conditional spacing: add space/comma directly to the value …
given = collectors['given'].fillna('')
# .where(condition, value_if_true) ensures spaces only appear when data is present
part = collectors['particle'].where(collectors['particle'].isna(), " " + collectors['particle'])
fam  = collectors['family'].where(collectors['family'].isna(), " " + collectors['family'])
suff = collectors['suffix'].where(collectors['suffix'].isna(), ", " + collectors['suffix'])
# 2. Vectorized concatenation
collectors['canonical_string_collector_parsed'] = (
    given + part.fillna('') + fam.fillna('') + suff.fillna('')
).str.strip()
# Optional: Clean up multiple spaces if they exist in the source data
collectors['canonical_string_collector_parsed'] = (
    collectors['canonical_string_collector_parsed'].str.replace(r'\s+', ' ', regex=True)
)

criterion = collectors["particle"].str.contains("\\w+ \\w+", na=False)

if explain_and_show_the_data: 
    # display(collectors['canonical_string_collector_parsed'][criterion].head())
    display(collectors[['canonical_string_collector_parsed', 'particle']][criterion].drop_duplicates().head(10))

combine parts of names similar to WikiData's given name labels


,canonical_string_collector_parsed,particle
52,H.A. van der Aa,van der
89,D.K. van der Aanen,van der
92,P.J.M. van der Aart,van der
542,A. van der Abdullah,van der
683,Thaj van der Abeleven,van der
760,H. van der Abé,van der
2096,C. van der Alders,van der
2099,K. van der Alders,van der
2731,J. J. M. van van der Alphen,van der
2837,W. van der Altenburg,van der


In [7]:
# move canonical_string_collector_parsed after column title (title was the last of the parsing columns)
col = collectors.pop("canonical_string_collector_parsed")
collectors.insert(collectors.columns.get_loc('title') + 1, col.name, col)

these_columns=["family", "given", "suffix", "particle", "dropping_particle", "nick", "appellation", "title", 'canonical_string_collector_parsed']

if 'source_data' in collectors.columns:
    these_columns.append("source_data")

display(collectors.tail().get(these_columns))

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data
189370,Štepánek,J.,NaN,NaN,NaN,NaN,NaN,NaN,J. Štepánek,Štepánek J; Trávnícek B
89855,Štepánek,J.,NaN,NaN,NaN,NaN,NaN,NaN,J. Štepánek,Kirschnerová L; Kirschner J; Štepánek J
189355,Štepánek,J.,NaN,NaN,NaN,NaN,NaN,NaN,J. Štepánek,Štepánek J; Jakl J
65698,Štepánek,J.,NaN,NaN,NaN,NaN,NaN,NaN,J. Štepánek,Hadinec J; Kirschner J; Sourkova M; Štepánek J
154419,Šumberová,K.,NaN,NaN,NaN,NaN,NaN,NaN,K. Šumberová,Simons ELAN; Šumberová K


In [8]:
# group and aggregate data to have unique name rows only for the matching of names later on
collectors_unique=collectors.groupby(['canonical_string_collector_parsed']).agg(
    family=('family', lambda x: list(x)[0]),
    given=('given', lambda x: list(x)[0]),
    suffix=('suffix', lambda x: list(x)[0]),
    particle=('particle', lambda x: list(x)[0]),
    dropping_particle=('dropping_particle', lambda x: list(x)[0]),
    nick=('nick', lambda x: list(x)[0]),
    appellation=('appellation', lambda x: list(x)[0]),
    title=('title', lambda x: list(x)[0]),
    source_data=('source_data', lambda x: list(x)[0]),
    occurrenceID_collectors_count= ('occurrenceID_count', 'sum'), # use count function
    occurrenceID_collectors_firstsample=('occurrenceID_firstsample', lambda x: list(x)[0]), # custom function, to get the first entry
    collectors_eventDate_mean=('eventDate_mean', 'mean'),
    collectors_eventDate_min=('eventDate_min', 'min'),
    collectors_eventDate_max=('eventDate_max', 'max')
).reset_index()

# move canonical_string_collector_parsed after column title
col = collectors_unique.pop("canonical_string_collector_parsed")
collectors_unique.insert(collectors_unique.columns.get_loc('title') + 1, col.name, col)
collectors_unique.drop_duplicates(inplace=True)

if explain_and_show_the_data: display(collectors_unique)

# column naming perhaps more clear (because we condensed the data)?
# collectors=collectors.add_suffix('_namegrouped') \
#  if not any(col.endswith("_namegrouped") for col in list(collectors.columns))

,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max
0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,A.,25,https://data.biodiversitydata.nl/naturalis/spe...,1980-11-15 18:19:38,1899-08-07,1999-12-10
1,A'buino'o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A'buino'o,A'buino'o; Hunt PF,1,https://data.biodiversitydata.nl/naturalis/spe...,1965-08-24 00:00:00,1965-08-24,1965-08-24
2,Hayek,A. A. von,NaN,von,NaN,NaN,NaN,NaN,A. A. von von Hayek,Hayek AA von; Hayek F von,1,https://data.biodiversitydata.nl/naturalis/spe...,1911-08-01 00:00:00,1911-08-01,1911-08-01
3,Aaku,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Aaku,Aaku A,3,https://data.biodiversitydata.nl/naturalis/spe...,1909-11-19 16:00:00,1909-05-28,1910-07-29
4,Aalders,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Aalders,Sterk AA; Aalders A,1,https://data.biodiversitydata.nl/naturalis/spe...,1975-06-20 00:00:00,1975-06-20,1975-06-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58759,Johansen,Ø.,NaN,NaN,NaN,NaN,NaN,NaN,Ø. Johansen,Johansen Ø,5,https://data.biodiversitydata.nl/naturalis/spe...,1976-11-16 04:48:00,1975-12-19,1978-02-07
58760,Weholt,Ø.,NaN,NaN,NaN,NaN,NaN,NaN,Ø. Weholt,Weholt Ø,6,https://data.biodiversitydata.nl/naturalis/spe...,1974-01-03 12:00:00,1927-01-01,1984-08-18
58761,Rustan,Ø.H.,NaN,NaN,NaN,NaN,NaN,NaN,Ø.H. Rustan,Rustan ØH,5,https://data.biodiversitydata.nl/naturalis/spe...,NaT,NaT,NaT
58762,Øcthagen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Øcthagen,Krog H; Øcthagen,1,https://data.biodiversitydata.nl/naturalis/spe...,1980-07-01 00:00:00,1980-07-01,1980-07-01


In [9]:
if explain_and_show_the_data:
    print("show collectors with highest occurrenceID_collectors_count")
    display(collectors_unique.sort_values(by=['occurrenceID_collectors_count', 'family', 'given'], ascending=[False, True, True]).head(10))

show collectors with highest occurrenceID_collectors_count


,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max
6691,Boom,B.K.,NaN,NaN,NaN,NaN,NaN,NaN,B.K. Boom,Ooststroom SJ van; Boom BK,51103,https://data.biodiversitydata.nl/naturalis/spe...,1958-05-10 04:33:59,1856-01-01,1997-04-11
17835,Breteler,F.J.,NaN,NaN,NaN,NaN,NaN,NaN,F.J. Breteler,Maas PJM; Breteler FJ; Maas-van de Kamer H; Ni...,41100,https://data.biodiversitydata.nl/naturalis/spe...,1988-10-25 10:05:30,1955-06-12,2020-03-06
30284,Maxwell,J.F.,NaN,NaN,NaN,NaN,NaN,NaN,J.F. Maxwell,Maxwell JF; Sankamethawee W,38782,https://data.biodiversitydata.nl/naturalis/spe...,1996-08-29 12:11:24,1969-01-18,2013-04-11
51351,Koorders,S.H.,NaN,NaN,NaN,NaN,NaN,NaN,S.H. Koorders,Koorders SH; Valeton T,34147,https://data.biodiversitydata.nl/naturalis/spe...,1917-12-17 06:47:23,1829-08-27,2012-11-11
3984,Leeuwenberg,A.J.M.,NaN,NaN,NaN,NaN,NaN,NaN,A.J.M. Leeuwenberg,Sidiyasa K; Leeuwenberg AJM; Arbainsyah,32591,https://data.biodiversitydata.nl/naturalis/spe...,1973-10-23 01:24:35,1926-02-20,1999-11-16
34101,Ajgh,Kostermans,NaN,NaN,NaN,NaN,NaN,NaN,Kostermans Ajgh,Kostermans AJGH; Soegeng-Reksodihardjo W,30712,https://data.biodiversitydata.nl/naturalis/spe...,1959-02-23 21:53:36,1892-09-30,1994-11-15
6530,Wilde-Duyfjes,B.E.E.,NaN,NaN,NaN,NaN,NaN,NaN,B.E.E. Wilde-Duyfjes,Mennema J; Wilde-Duyfjes BEE de,29893,https://data.biodiversitydata.nl/naturalis/spe...,1986-10-15 13:20:06,1958-06-28,2019-09-04
52577,Itinere,Stud,NaN,biol Rheno-Trai in,NaN,NaN,NaN,NaN,Stud biol Rheno-Trai in Itinere,Stud biol Rheno-Trai in itinere; Krüger JHJ,28222,https://data.biodiversitydata.nl/naturalis/spe...,1963-03-31 08:11:10,1847-06-18,1996-07-08
31148,Soest,J.L.,NaN,van,NaN,NaN,NaN,NaN,J.L. van Soest,Unknown; Soest JL van,25106,https://data.biodiversitydata.nl/naturalis/spe...,1927-09-01 15:50:20,1803-08-10,1994-09-01
30907,Wieringa,J.J.,NaN,NaN,NaN,NaN,NaN,NaN,J.J. Wieringa,Wieringa JJ,22913,https://data.biodiversitydata.nl/naturalis/spe...,2006-10-31 04:52:04,1980-08-19,2022-11-12


In [10]:
# Idea: Should we use data column suffixes to follow the data source after merging is done later?
# collectors=collectors.add_suffix('_parsed') \
#  if not any(col.endswith("_parsed") for col in list(collectors.columns))

### Set Up the Text Analysis

See https://towardsdatascience.com/fuzzy-matching-at-scale-84f2bfd0c536 for deeper understanding.

The `ngrams` function is used as an analyzer in the text search later.

In [11]:
import re
import unicodedata
!pip install ftfy # amazing text cleaning for decode issues..
from ftfy import fix_text

# Regex for unwanted special characters
CHARS_TO_REMOVE_RX = re.compile(r"[)(.|\[\]{}'„“”\"‚‘’›‹»«]|[,-./]|\sBD")
MULTIPLE_SPACES_RX = re.compile(r' +')

def ngrams(string, n=3):
    """
    Construct ngram(s) of a given text
     
    @param string: the text string to perform the ngram splitting on 
    @param n: character length of the particular (split) result text each
    @return: string as ngram
    """
    if not string:
        return []

    # (fix encoding errors)
    string = fix_text(string)

    # Normalization (IMPORTANT for Unicode)
    # NFC ensures that characters such as ‘é’ are treated as a single character
    # rather than as a combination of ‘e’ + accent.
    string = unicodedata.normalize('NFC', string)
    string = string.lower()    
    # (other languages often use different separators; kept generic here)
    string = string.replace('&', 'and').replace(',', ' ').replace('-', ' ')    
    # character cleanup but leave then as spaces
    string = CHARS_TO_REMOVE_RX.sub(' ', string)    
    # Normalization of spaces & case: ….title() also works with umlauts (e.g., “öllegard” -> “Öllegard”)
    string = string.title()
    string = MULTIPLE_SPACES_RX.sub(' ', string).strip()    
    # Padding & N-gram generation
    string = f" {string} "
    return [string[i : i + n] for i in range(len(string) - n + 1)]

In [12]:
import numpy as np
import time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

def calculateTFIDFmatchingOfData(query_data, match_data, n_neighbors=1):
    """
    Calculate a TF-IDF (Term Frequency — Inverse Document Frequency) matching with getNearestN

    @param query_data: DataFrame usually a pandas data column to query names or strings for
    @param match_data: DataFrame against to match with
    @param n_neighbors: number of similar data neighbors to calculate for (output as well; 5 = 5 times more output data, therefore default: 1)

    @requires NearestNeighbors()
    @requires getNearestNeighbour()
    @requires ngrams()
    @requires TfidfVectorizer()
    @requires NearestNeighbors()

    @return: DataFrame a data frame of matches with columns 'namematch_source_data', 'namematch_resource_data', 'namematch_distance'
    """
    # TODO if n_neighbors > 1 then output data need more explaining columns, 
    # or ensure the right ordering, because as e.g. n_neighbors=5  
    # then 5 sub-samples of the same name shall be exist
    
    start = time.time()
    
    # Unique values for performance, but directly as a list for stable index
    query_list = list(set(query_data))
    match_values = match_data.values # Cache für schnellen Zugriff
    
    print(f'Vectorizing {len(match_data)} reference items...')
    # Tip: ‘ngrams’ must be defined globally or appear here as a lambda.
    vectorizer = TfidfVectorizer(min_df=1, analyzer=ngrams, lowercase=False)
    tfidf_match_matrix = vectorizer.fit_transform(match_data)
    
    nbrs = NearestNeighbors(n_neighbors=n_neighbors, n_jobs=-1, metric='cosine').fit(tfidf_match_matrix)
    
    print(f'Transforming {len(query_list)} queries...')
    tfidf_query_matrix = vectorizer.transform(query_list)
    
    # Start search
    distances, indices = nbrs.kneighbors(tfidf_query_matrix)
    
    print('Building DataFrame...')
    
    # We repeat each entry in query_list n_neighbors times
    # so that the column lengths for the DataFrame match
    query_repeated = np.repeat(query_list, n_neighbors)
    # TODO SUBSAMPLES n_neighbors: query_ids = np.repeat(np.arange(len(query_list)), n_neighbors)
    
    # Ensure that match_values is a flat NumPy array
    # .ravel() is the counterpart to .flatten(), but works more reliably
    flat_indices = indices.ravel()
    
    # We access the values and ensure that we have a NumPy format.
    match_hits = np.array(match_values)[flat_indices]
    
    matches = pd.DataFrame({
        # TODO SUBSAMPLES n_neighbors: 'group_index_id': query_ids,
        'namematch_source_data': query_repeated,
        'namematch_resource_data': match_hits,
        'namematch_distance': distances.ravel(),
        # TODO SUBSAMPLES n_neighbors: 'namematch_index': flat_indices
    })
    
    # Optional: Rounding the distance
    matches['namematch_distance'] = matches['namematch_distance'].round(4)

    print(f'Done in {time.time() - start:.2f}s')
    return matches

In [13]:
# some example data
samples = [
    ("simple name", "Klazenga, N."),
    ("simple name", "金井弘夫(Hiroo Kanai)"),
    ("simple name", "Н. А. Антипова"),
    ("data from collectors", collectors_unique["canonical_string_collector_parsed"].at[1]),
    ("data from match-test", collectors_unique["canonical_string_collector_parsed"].at[1]),
    ("data from match-test, full names", wd_matchtest_fullnames['canonical_string_fullname'].at[0])
]

print("Show ngram examples:")
for label, name in samples:
    print(f"- {label} “{name}”:", ngrams(name))

# some example data
print('\n(WikiData’s) canonical_string = (constructed) canonical_string_fullname:')
# We only take the first 5 lines of both columns.
short_names = wd_matchtest['canonical_string'].head(5)
long_names = wd_matchtest_fullnames['canonical_string_fullname'].head(5)

for short, long in zip(short_names, long_names):
    print(f"- {short} = {long}")

Show ngram examples:
- simple name “Klazenga, N.”: [' Kl', 'Kla', 'laz', 'aze', 'zen', 'eng', 'nga', 'ga ', 'a N', ' N ']
- simple name “金井弘夫(Hiroo Kanai)”: [' 金井', '金井弘', '井弘夫', '弘夫 ', '夫 H', ' Hi', 'Hir', 'iro', 'roo', 'oo ', 'o K', ' Ka', 'Kan', 'ana', 'nai', 'ai ']
- simple name “Н. А. Антипова”: [' Н ', 'Н А', ' А ', 'А А', ' Ан', 'Ант', 'нти', 'тип', 'ипо', 'пов', 'ова', 'ва ']
- data from collectors “A'buino'o”: [' A ', 'A B', ' Bu', 'Bui', 'uin', 'ino', 'no ', 'o O', ' O ']
- data from match-test “A'buino'o”: [' A ', 'A B', ' Bu', 'Bui', 'uin', 'ino', 'no ', 'o O', ' O ']
- data from match-test, full names “"Fritz" Ryser”: [' Fr', 'Fri', 'rit', 'itz', 'tz ', 'z R', ' Ry', 'Rys', 'yse', 'ser', 'er ']

(WikiData’s) canonical_string = (constructed) canonical_string_fullname:
- "F." Ryser = "Fritz" Ryser
- "N.A. Antipova" (lapsus) = "N.A. Antipova" (lapsus)
- "N.A.Antipova" (lapsus) = "N.A.Antipova" (lapsus)
- "The grandmother of female scientists in Ghana" = "The grandmother of fe

Vectorize Wikidata names. Background: We use an information retrieval technique (Term Frequency — Inverse Document Frequency, blog [towardsdatascience.com/tf-idf-explained…](https://towardsdatascience.com/tf-idf-explained-and-python-sklearn-implementation-b020c5e83275)) for matching the source names with WikiData names, for that a calculated dinsance measure of the name match will help to match similar names and distinguish names that are rather no match. In general see also https://scikit-learn.org, https://pypi.org/project/scikit-learn/.

### Perform the Matching

Perform the nearest neighbour (NN) matches on the (Naturalis) collector names and create a data frame with matches, and we try to distinguish abbreviated and full names in the source to better match source data and WikiData ... (can take 5 to 10 minutes)

Now convert a collection of raw documents to a matrix of TF-IDF features and set up the function that performs the matches...

In [14]:
criterion_fullnames = collectors_unique.given.str.contains('^\\w{3,}', na=False)
collectors_names = collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values
# collectors_names = set(collectors_unique['canonical_string_collector_parsed'][[not fullname for fullname in criterion_fullnames]].values)
print("Calculate rather the abbreviated names only …")
matches = calculateTFIDFmatchingOfData(collectors_names, wd_matchtest['canonical_string'])

matches = matches.sort_values(['namematch_distance'])
matches = matches.reset_index(names=['old_index'])

if explain_and_show_the_data: display(matches)

Calculate rather the abbreviated names only …
Vectorizing 171448 reference items...
Transforming 56725 queries...
Building DataFrame...
Done in 250.70s


,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,38568,W.A. Talbot,W. A. Talbot,0.0000
1,12927,Chitr,Chitr.,0.0000
2,12922,M. Atahuachi,M. Atahuachi,0.0000
3,12912,Andrée,Andrée,0.0000
4,38575,D. Franklin.,D. Franklin,0.0000
...,...,...,...,...
56720,55912,T. Oikmoet,Kloet,0.7531
56721,48213,Luwiik,M. Struwig,0.7541
56722,21928,Mangoenkromodiredjo,Mangold,0.7561
56723,29965,Ndutumu,Peduto,0.7568


In [15]:
# criterion_fullnames = collectors_unique.given.str.contains('^\\w{3,}', na=False)
collectors_fullnames = collectors_unique['canonical_string_collector_parsed'][criterion_fullnames].values

print("Calculate rather the full names only …")
matches_fullnames = calculateTFIDFmatchingOfData(collectors_fullnames, wd_matchtest_fullnames['canonical_string_fullname'])

matches_fullnames = matches_fullnames.sort_values(['namematch_distance'])
matches_fullnames = matches_fullnames.reset_index(names=['old_index'])

if explain_and_show_the_data: display(matches_fullnames)

Calculate rather the full names only …
Vectorizing 204793 reference items...
Transforming 2039 queries...
Building DataFrame...
Done in 25.51s


,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,568,Den Berger,Den Berger,0.0000
1,21,Pertti Uotila,Pertti Uotila,0.0000
2,58,Nguyen Kim Dao,Nguyen Kim Dao,0.0000
3,1695,"King Leopold, III",King Leopold III,0.0000
4,101,Flora Martin,Flora Martin,0.0000
...,...,...,...,...
2034,1429,Instituut voor Prehistorie Leiden,Van Vooren,0.7489
2035,380,Instituut Veredeling Tuinbouwgewassen,Vered,0.7536
2036,1643,Lab Palyn Ephe-Cnrs,CNS,0.7549
2037,749,Landbouwleraar van Poerwodredjo,Grandbois,0.7724


### Create Output Results

Combine the matches data frame back to the (Naturalis) collectors and Wikidata items …

In [16]:
if explain_and_show_the_data: print("join matches data frame back to source collectors dataframe")
collectors_matches = pd.merge(
    collectors_unique, matches, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

if explain_and_show_the_data: display(collectors_matches.head())

join matches data frame back to source collectors dataframe


,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A,A.,25,https://data.biodiversitydata.nl/naturalis/spe...,1980-11-15 18:19:38,1899-08-07,1999-12-10,40369,A,A. A.,0.2696
1,A'buino'o,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A'buino'o,A'buino'o; Hunt PF,1,https://data.biodiversitydata.nl/naturalis/spe...,1965-08-24 00:00:00,1965-08-24,1965-08-24,45082,A'buino'o,Buining,0.5149
2,Hayek,A. A. von,NaN,von,NaN,NaN,NaN,NaN,A. A. von von Hayek,Hayek AA von; Hayek F von,1,https://data.biodiversitydata.nl/naturalis/spe...,1911-08-01 00:00:00,1911-08-01,1911-08-01,31587,A. A. von von Hayek,"N., A. von",0.3743
3,Aaku,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Aaku,Aaku A,3,https://data.biodiversitydata.nl/naturalis/spe...,1909-11-19 16:00:00,1909-05-28,1910-07-29,35585,A. Aaku,Kaku,0.4229
4,Aalders,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Aalders,Sterk AA; Aalders A,1,https://data.biodiversitydata.nl/naturalis/spe...,1975-06-20 00:00:00,1975-06-20,1975-06-20,49029,A. Aalders,Aalders,0.0565


In [17]:
if explain_and_show_the_data: print("show full name matches, and append them to all matches")
collectors_matches_fullname = pd.merge(
    collectors_unique, matches_fullnames, 
    left_on='canonical_string_collector_parsed', right_on='namematch_source_data'
    #, suffixes=(None, '_namematch') # append to left-data, right-data only when identical column names occur
)

if explain_and_show_the_data: display(collectors_matches_fullname.head())

show full name matches, and append them to all matches


,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_distance
0,Bergh Mh,Aarts-van,NaN,den,NaN,NaN,NaN,NaN,Aarts-van den Bergh Mh,Aarts-van den Bergh MH,2,https://data.biodiversitydata.nl/naturalis/spe...,1970-04-02 00:00:00,1950-09-01,1989-11-01,487,Aarts-van den Bergh Mh,G. van den Bergh,0.3732
1,Adan,Abar,NaN,bin,NaN,NaN,NaN,NaN,Abar bin Adan,Mantri; Abar bin Adan,39,https://data.biodiversitydata.nl/naturalis/spe...,1922-06-12 19:24:42,1919-08-02,1929-12-03,1616,Abar bin Adan,Adan,0.4771
2,Saidi,Abas Gelar,NaN,NaN,NaN,NaN,NaN,NaN,Abas Gelar Saidi,Abas Gelar St. Saidi,2,https://data.biodiversitydata.nl/naturalis/spe...,1933-05-05 00:00:00,1933-05-05,1933-05-05,104,Abas Gelar Saidi,Gelardi,0.5126
3,Des,Abbayes H.R.N.,NaN,NaN,NaN,NaN,NaN,NaN,Abbayes H.R.N. Des,Abbayes HRN des; Stomps TJ,11,https://data.biodiversitydata.nl/naturalis/spe...,1941-04-23 06:24:00,1937-07-10,1948-11-22,316,Abbayes H.R.N. Des,Abbayes,0.2857
4,Ms-A,Abdallah,NaN,NaN,NaN,NaN,NaN,NaN,Abdallah Ms-A,Abdallah MS-A; Sa'ad FE-ZM; Mahdy M; Abbas A,393,https://data.biodiversitydata.nl/naturalis/spe...,1965-08-15 03:58:05,1936-02-11,1985-01-18,1081,Abdallah Ms-A,Abdallah,0.2517


In [18]:
collectors_all_matches=pd.concat([collectors_matches, collectors_matches_fullname])
collectors_all_matches.sort_values(by=['namematch_distance', 'family'], ascending=[True, True], inplace=True)
if explain_and_show_the_data:
    print("show match results of all abbreviated and full names")
    display(collectors_all_matches.head())

show match results of all abbreviated and full names


,family,given,suffix,particle,dropping_particle,nick,appellation,title,canonical_string_collector_parsed,source_data,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,collectors_eventDate_mean,collectors_eventDate_min,collectors_eventDate_max,old_index,namematch_source_data,namematch_resource_data,namematch_distance
4311,A.M.C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A.M.C,Wiakabu J; A.M.C.,112,https://data.biodiversitydata.nl/naturalis/spe...,1980-08-17 14:47:08,1980-07-22,1980-09-11,33075,A.M.C,A.M.C.,0.0
12186,Aanen,D.K.,NaN,NaN,NaN,NaN,NaN,NaN,D.K. Aanen,Aanen DK; Arnolds EJM,35,https://data.biodiversitydata.nl/naturalis/spe...,1996-01-30 13:36:00,1995-09-15,1996-10-20,17162,D.K. Aanen,D. K. Aanen,0.0
5,Aaronsohn,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Aaronsohn,Aaronsohn A,3,https://data.biodiversitydata.nl/naturalis/spe...,1907-01-26 12:00:00,1906-12-06,1907-03-19,15243,A. Aaronsohn,A. Aaronsohn,0.0
10,Abbas,A.,NaN,NaN,NaN,NaN,NaN,NaN,A. Abbas,Abdallah MS-A; Sa'ad FE-ZM; Mahdy M; Abbas A,378,https://data.biodiversitydata.nl/naturalis/spe...,1963-03-03 08:38:53,1936-02-11,1963-11-01,33102,A. Abbas,A. Abbas,0.0
4800,Abbe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abbe,Abbe,3,https://data.biodiversitydata.nl/naturalis/spe...,1928-01-01 00:00:00,1928-01-01,1928-01-01,52898,Abbe,Abbe,0.0


In [19]:
# criterion = collectors_all_matches['canonical_string_collector_parsed'].map(lambda x: x.startswith('Kotschy'))
# print("Show example of «Kotschy…» with namematch distances from 0.0 to 1.0 (in Cosine Similiarity we had 0.5 … 1.0)")
# collectors_all_matches[criterion]

In [20]:
## cell split - moarkdown
# Save the plain name matching results only ...
## cell split - code
# if not os.path.exists('data'):
#     print("Make data directory for saving …")
#     os.makedirs('data')
# 
# # Set some global varialbes
# # this_timestamp_for_data=time.strftime('%Y%m%d') # 20230719
# this_timestamp_for_data=20231030
# 
# this_output_file='data/results_naturalis_collectors-eventDate_vs_wikidata-botanists_kneighbor_plain-names_%s.csv' % (
#     this_timestamp_for_data
# )
# 
# collectors_all_matches.to_csv(this_output_file)
# 
# print("Wrote plain name matches of collector names into %s (%d kB)" % 
#     (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
# )

### Merge Matched Data and WikiData’s

Review (TODO)
- evaluate time references: `eventDate` ~ `yob`, `wyb`—perhaps define a score value that could integrate all scores from properties we need for decision of the name matching (name distance, eventDate ~ year of birth/work year begin aso.)
- merge abbreviated and full name data properly, distinguish abbrevited match and full name match
- refactor `collectors_matches` or `collectors_matches_g1` aso. to `collectors_all_matches`
- refactor `collectors` to `collectors_unique`
- refactor `matches`to `matches_abbr` or distinguish `matches_fullname`

Now
1. merge now the matching data and the wiki data’s on the conaonical string name
2. later aggregate fine tuned, checking if multiple same (canonical string) names relate to multiple different persons (we use wd-items (the Q1233242 thing), and wd-item-labels to aggregate on) … aso.
3. save those data tables

In [21]:
if explain_and_show_the_data: print("merge now the matching data and the wiki data’s on the conaonical string name")
    
collectors_matches_tmp_names_abbreviated = pd.merge(
    collectors_matches, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string'
)
collectors_matches_tmp_fullnames = pd.merge(
    collectors_matches_fullname, wikidata, 
    left_on='namematch_resource_data', right_on='canonical_string_fullname'
)
collectors_matches_g1_merged_wikidata = pd.concat(
    [collectors_matches_tmp_names_abbreviated, collectors_matches_tmp_fullnames]
    , ignore_index=True
)

merge now the matching data and the wiki data’s on the conaonical string name


In [28]:
if explain_and_show_the_data:
    print("Show some name match examples …")
    for testname in ['Louis', 'Abbot']:
        print(f"Show e.g. «{testname}…» matching names …")
        criterion = collectors_matches_g1_merged_wikidata['canonical_string_collector_parsed'].str.contains(testname)
        this_table=collectors_matches_g1_merged_wikidata[criterion].get([
            # 'canonical_string_collector_parsed', # canonical_string_collector_parsed = namematch_source_data
            'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
            'namematch_source_data', 'namematch_resource_data', 'namematch_distance', 
            'itemLabel', 
            'canonical_string_fullname', # canonical_string_fullname contains the former itemMatchingLabel
            'wikidata_link',
            'collectors_eventDate_min', 'collectors_eventDate_max',
            'yob', 'yod',            
        ]).sort_values(by=['namematch_distance'], ascending=[True])
        print("# ---------------------------------------------\n# «%s…» as test name, %d collector names contain:" % (testname, criterion.sum()))    
        display(this_table)

Show some name match examples …
Show e.g. «Louis…» matching names …
# ---------------------------------------------
# «Louis…» as test name, 18 collector names contain:


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_distance,itemLabel,canonical_string_fullname,wikidata_link,collectors_eventDate_min,collectors_eventDate_max,yob,yod
1847,2,https://data.biodiversitydata.nl/naturalis/spe...,A. Louis,A.Louis,0.0000,A. Louis,A.Louis,http://www.wikidata.org/wiki/Q33682458,NaT,NaT,<NA>,<NA>
5344,10542,https://data.biodiversitydata.nl/naturalis/spe...,A.M. Louis,A.M.Louis,0.0000,Adriaan M. Louis,A.M.Louis,http://www.wikidata.org/wiki/Q21338327,1969-04-10,2013-03-02,1944,<NA>
38378,3339,https://data.biodiversitydata.nl/naturalis/spe...,J.L.P. Louis,J. L. P. Louis,0.0000,Jean Laurent Prosper Louis,Jean Laurent Prosper Louis,http://www.wikidata.org/wiki/Q5928759,1900-01-01,1998-05-17,1903,1947
27486,3,https://data.biodiversitydata.nl/naturalis/spe...,H. Louis,H. Louis,0.0000,Herbert Louis Mason,Herbert Louis,http://www.wikidata.org/wiki/Q11925452,1907-06-01,1953-10-01,1896,1994
44979,59,https://data.biodiversitydata.nl/naturalis/spe...,Louis,Louis,0.0000,Jean Laurent Prosper Louis,Louis,http://www.wikidata.org/wiki/Q5928759,1904-05-28,1984-09-21,1903,1947
72493,10,https://data.biodiversitydata.nl/naturalis/spe...,Louis-Marie Lalonde,Louis-Marie Lalonde,0.0000,Louis-Marie Lalonde,Louis-Marie Lalonde,http://www.wikidata.org/wiki/Q5981449,1926-07-15,1942-07-09,1896,1978
53790,51,https://data.biodiversitydata.nl/naturalis/spe...,P. Louis-Marie,Louis-Marie,0.0577,Louis-Marie Lalonde,Louis-Marie,http://www.wikidata.org/wiki/Q5981449,1925-07-28,1953-07-08,1896,1978
44980,6,https://data.biodiversitydata.nl/naturalis/spe...,Louis-Arsène,L. F. Louis-Arsène,0.0656,Louis Arsène,Le Frère Louis-Arsène,http://www.wikidata.org/wiki/Q21505357,1927-04-29,1936-08-01,1875,1959
21897,116,https://data.biodiversitydata.nl/naturalis/spe...,F.H. Louis,H. Louis,0.0821,Herbert Louis Mason,Herbert Louis,http://www.wikidata.org/wiki/Q11925452,1853-08-03,1960-09-25,1896,1994
20946,1,https://data.biodiversitydata.nl/naturalis/spe...,F. Pierre-Louis,S. Pierre-Louis,0.0966,Marie-Thérèse Kerger,Soeur Pierre-Louis,http://www.wikidata.org/wiki/Q62990315,1953-12-01,1953-12-01,1927,2013


Show e.g. «Abbot…» matching names …
# ---------------------------------------------
# «Abbot…» as test name, 19 collector names contain:


,occurrenceID_collectors_count,occurrenceID_collectors_firstsample,namematch_source_data,namematch_resource_data,namematch_distance,itemLabel,canonical_string_fullname,wikidata_link,collectors_eventDate_min,collectors_eventDate_max,yob,yod
5977,14,https://data.biodiversitydata.nl/naturalis/spe...,A.T.D. Abbott,A. T. D. Abbott,0.0000,A. T. D. Abbott,A. T. D. Abbott,http://www.wikidata.org/wiki/Q117328147,1997-02-13,2010-05-27,1936,2013
6139,1,https://data.biodiversitydata.nl/naturalis/spe...,Abbott,Abbott,0.0000,George Abbott,Abbott,http://www.wikidata.org/wiki/Q47112598,NaT,NaT,<NA>,<NA>
15642,1,https://data.biodiversitydata.nl/naturalis/spe...,D.P. Abbott,D. P. Abbott,0.0000,Donald Putnam Abbott,Donald Putnam Abbott,http://www.wikidata.org/wiki/Q21663971,1967-08-02,1967-08-02,1920,1986
19131,2,https://data.biodiversitydata.nl/naturalis/spe...,E.K. Abbott,E. K. Abbott,0.0000,Edwin Kirk Abbott,E. K. Abbott,http://www.wikidata.org/wiki/Q81587932,1889-01-01,1889-04-01,1840,1918
19132,2,https://data.biodiversitydata.nl/naturalis/spe...,E.K. Abbott,E. K. Abbott,0.0000,Edwin Kirk Abbott,Edwin Kirk Abbott,http://www.wikidata.org/wiki/Q81587932,1889-01-01,1889-04-01,1840,1918
19133,2,https://data.biodiversitydata.nl/naturalis/spe...,E.K. Abbott,E. K. Abbott,0.0000,Edwin Kirk Abbott,Erwin Kirk Abbott,http://www.wikidata.org/wiki/Q81587932,1889-01-01,1889-04-01,1840,1918
31339,106,https://data.biodiversitydata.nl/naturalis/spe...,I.A. Abbott,I. A. Abbott,0.0000,Isabella Abbott,I. A. Abbott,http://www.wikidata.org/wiki/Q6077932,1946-05-01,1995-02-22,1919,2010
31340,106,https://data.biodiversitydata.nl/naturalis/spe...,I.A. Abbott,I. A. Abbott,0.0000,Isabella Abbott,Isabella A. Abbott,http://www.wikidata.org/wiki/Q6077932,1946-05-01,1995-02-22,1919,2010
31341,106,https://data.biodiversitydata.nl/naturalis/spe...,I.A. Abbott,I. A. Abbott,0.0000,Isabella Abbott,Isabella Aiona Abbott,http://www.wikidata.org/wiki/Q6077932,1946-05-01,1995-02-22,1919,2010
69468,10,https://data.biodiversitydata.nl/naturalis/spe...,W.L. Abbott,W. L. Abbott,0.0000,William Louis Abbott,W L Abbott,http://www.wikidata.org/wiki/Q635604,1922-04-05,1922-04-30,1860,1936


## Write DarwinCore Attribution Output

Here we map table data fields to fields of DarwinCore Attribution (<https://github.com/tdwg/attribution/>, <https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml>) 

### Scoring

Individual scored properties should actually be balanced in such a way that one can simply add up these different property scores; in this case, assessment of the calculated values is still necessary. The problem here with calculation with a distance measure is that we have the opposite of similarity, whose distance can become greater than 1, which must somehow be mapped to a scope of 0 … 1 (or -1 … 0 … 1) (TODO review).

General thoughts: With a score of -1 to 1, it can be assumed that:
* -1 means full devaluation or no agreement
* 1 means full upvoting or agreement, and
* 0 can have several interpretations: it is in between, or no rating possible, or missing values.

### Task to Be Solved in Evaluating the Life Time ~ Rating/Scoring

We have grouped the collection date (evenDate) to the name in the source data, so it may be that for (abbreviated) names, e.g. “Bachmann, F.”, the collection date is valid for *several* personal names, not just one. This must be taken into account when considering and evaluating whether the life data match the collection date. The rating of the life data has the following idea:

| Score (life time) | Remarks | 
|--|--|
| 1.0  | complete match                     |
| 0.5  | somewhat correct, but has errors or mistakes, indicating multiple person names    |
| 0.0   | no evaluation (or not possible) |
| -0.5 | is rather to be rejected, indicating multiple person names and possibly overlapping time spans of the collection date of different person names, or mistakes in the original data |
| -1.0 | completely rejected                |

### Task to Be Solved With Several Names ~ Assessment/Score

Since we do not know if there are other possible names somewhere when there is only one name, we cannot assign a “1” (= full agreement) with certainty, so it was decided that if only 1 name was found, this would be evaluated as zero, in the sense of no evaluation. So when evaluating the multiple names, only the mismatches are evaluated, according to the idea:

| Score (multiple names) | Remarks | 
|--|--|
| 1.0  | this value (=full upvoting or agreement) would never be set in this regard, since we do not know all the full names of the cosmos ;-), and could state this score certainty of 1.0 |
| 0.0 | no evaluation, because only 1 name found | 
| less than 0 | multiple names found, i.e. deduction (perhaps just -0.5, as a decision needs to be made) | 

---

TODO review interpretation:

- the fields are defined in <https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml> and regarding from this DwC-attribution concept: is it correct to map it like the following (`name` would represent the *interpreted* resource name (in long format), not the *source* collector `name` (in (theoretically) long format))?
    ```
    name          ← itemLabel (wikiData)
    alternateName ← canonical_string_collector_parsed (actual collector name)
    ```

In [23]:
# TODO further evaluation or filtering, counting, clean up aso.
pprint.pprint(collectors_matches_g1_merged_wikidata.columns)

Index(['family', 'given', 'suffix', 'particle', 'dropping_particle', 'nick',
       'appellation', 'title', 'canonical_string_collector_parsed',
       'source_data', 'occurrenceID_collectors_count',
       'occurrenceID_collectors_firstsample', 'collectors_eventDate_mean',
       'collectors_eventDate_min', 'collectors_eventDate_max', 'old_index',
       'namematch_source_data', 'namematch_resource_data',
       'namematch_distance', 'item', 'itemLabel', 'surname', 'initials',
       'canonical_string', 'canonical_string_fullname', 'orcid', 'viaf',
       'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 'yob', 'yod',
       'wikidata_link', 'orcid_link', 'harv_link', 'ipni_link',
       'bionomia_link'],
      dtype='str')


In [24]:
# refactor namematch_similarity → namematch_distance
# refactor namematch_similarity_annotation → namematch_distance_annotation
# refactor custom_namematch_similarity → custom_namematch_namematch
# refactor sort_values
# refactor collectors_eventDate_mean
# refactor collectors_eventDate_min
# - refactor yob_is_lt_eventDate_min
# refactor collectors_eventDate_max
# - refactr yod_is_gt_eventDate_max
# refactor custom_score_lifetime            → custom_score_lifetime_data
# refactor custom_score_lifetime_annotation → custom_score_lifetime_data_annotation
# refactor namematch_similarity             → namematch_distance
# refactor namematch_similarity_annotation  → namematch_distance_annotation
# refactor custom_namematch_similarity → custom_namematch_namematch
# refactor sort_values
collectors_wikidata_cossimOrKmeans = collectors_matches_g1_merged_wikidata[
    [
        'canonical_string_collector_parsed', 'family', 'given', 
        'occurrenceID_collectors_count', 'occurrenceID_collectors_firstsample',
        'source_data',
        'namematch_source_data', 'namematch_resource_data', 'namematch_distance', 
        'item', 'canonical_string', 'itemLabel',
        'orcid', 'viaf', 'isni', 'harv', 'ipni', 'abbr', 'bionomia_id', 
        'collectors_eventDate_mean', 'collectors_eventDate_min', 'collectors_eventDate_max',
        'yob', 'yod',
        # 'wyb'
    ]
].copy()

# order by canonical_string_collector_parsed (actual collector name) (asc)
#   order by similarity (desc) or namematch_distance (asc), 
#     order by number of Wikidata items (asc) and 
#       order by number of collections (desc)
collectors_wikidata_cossimOrKmeans.sort_values(
    by=['canonical_string_collector_parsed', 'namematch_distance', 'family', 'given'], 
    ascending=[True, True, True, True], 
    inplace=True
)

dwcagent_attr_output=collectors_wikidata_cossimOrKmeans.get([
    "occurrenceID_collectors_firstsample", 
    "canonical_string_collector_parsed",
    'family', 'given',
    "namematch_distance", 
    "source_data", 
    "itemLabel", 
    "item",
    "collectors_eventDate_min",
    "collectors_eventDate_max",
    'yob', 'yod'
]).copy().drop_duplicates(ignore_index=True)


dwcagent_attr_output['canonical_string_collector_parsed'] = dwcagent_attr_output['canonical_string_collector_parsed'].astype(object)
dwcagent_attr_output['canonical_string_collector_parsed'] = dwcagent_attr_output['canonical_string_collector_parsed'].replace(
    to_replace=r'([^,]+),\s*(.+)',
    value=r'\\2 \\1',
    regex=True
)
dwcagent_attr_output['namematch_distance_annotation'] = dwcagent_attr_output['namematch_distance'].astype(str).str.replace(r'(.+)', '\\1 (k-means distance)', regex=True)
# dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'namematch_distance_annotation', '', allow_duplicates=True)

dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'life_time_periode', '', allow_duplicates=True)

combine_life_times = lambda this_df: ("%s-%s" % (this_df["yob"], this_df["yod"])).replace(r"<NA>", "?")
dwcagent_attr_output["life_time_periode"]=dwcagent_attr_output.apply(combine_life_times, axis="columns")

# dwcagent_attr_output["life_time_periode"]

years_from_birth_until_first_collection_activity = 10
dwcagent_attr_output["yob_is_lt_eventDate_min"] = dwcagent_attr_output["yob"] + years_from_birth_until_first_collection_activity < dwcagent_attr_output["collectors_eventDate_min"].dt.year
dwcagent_attr_output["yod_is_gt_eventDate_max"] = dwcagent_attr_output["yod"] > dwcagent_attr_output["collectors_eventDate_max"].dt.year
dwcagent_attr_output["custom_score_lifetime_data"] = 0.0
dwcagent_attr_output.insert(len(dwcagent_attr_output.columns), 'custom_score_lifetime_data_annotation', '', allow_duplicates=True)

import numpy as np

# 1. Variables - Ensure they are converted to standard numpy booleans
# We fill NA with False for the logical check to satisfy numpy.select
y_to_activity = years_from_birth_until_first_collection_activity 

b_ok = dwcagent_attr_output["yob_is_lt_eventDate_min"].fillna(False).astype(bool)
d_ok = dwcagent_attr_output["yod_is_gt_eventDate_max"].fillna(False).astype(bool)

# To still detect the "NA" cases in your conditions, we need the original NA info:
b_isna = dwcagent_attr_output["yob_is_lt_eventDate_min"].isna()
d_isna = dwcagent_attr_output["yod_is_gt_eventDate_max"].isna()

# 2. Updated Conditions (using the sanitized booleans and the NA-checkers)
conditions = [
    (b_ok == True)  & (d_ok == True),   # Both match
    (b_ok == True)  & (d_isna),         # Birth matches, death unknown
    (b_isna)        & (d_ok == True),   # Birth unknown, death matches
    (b_isna)        & (d_isna),         # Both unknown
    (b_ok == True)  & (d_ok == False) & (~d_isna),  # Birth matches, death contradicts
    (b_ok == False) & (~b_isna) & (d_ok == True),   # Birth contradicts, death matches
    (b_ok == False) & (~b_isna) & (d_isna),         # Birth contradicts, death unknown
    (b_isna)        & (d_ok == False) & (~d_isna),  # Birth unknown, death contradicts
    (b_ok == False) & (~b_isna) & (d_ok == False) & (~d_isna) # Both contradict
]

scores = [1.0, 1.0, 1.0, 0.0, 0.5, 0.5, -0.5, -0.5, -1.0]

labels = [
    "full match",
    "OK? year of death is missing",
    "OK? year of birth is missing",
    "unknown life time",
    f"OK yob + {y_to_activity}, but yod not matching, check name and lifetime data",
    f"yob + {y_to_activity} not matching, OK yod, check name and lifetime data",
    f"yob + {y_to_activity} not matching, yod unknown, check name and lifetime data",
    f"yob unknown, yod not matching, check name and lifetime data",
    f"life time not matching any eventDate (yob + {y_to_activity} … yod)"
]

# 3. Apply Scoring and Annotations
dwcagent_attr_output["custom_score_lifetime_data"] = np.select(conditions, scores, default=0.0)
dwcagent_attr_output["custom_score_lifetime_data_annotation"] = np.select(conditions, labels, default="check data consistency")

# 4. Handle Ambiguity
is_duplicated = dwcagent_attr_output['canonical_string_collector_parsed'].duplicated(keep=False)
dwcagent_attr_output["custom_score_multiple_names"] = np.where(is_duplicated, -0.5, 0.0)

# 5. Overall Score Calculation
namematch_distance_max = dwcagent_attr_output['namematch_distance'].max()
similarity = 1.0 - (dwcagent_attr_output['namematch_distance'] / namematch_distance_max) if namematch_distance_max > 0 else 1.0

reliability_subtotal = dwcagent_attr_output[["custom_score_lifetime_data", "custom_score_multiple_names"]].mean(axis=1)
dwcagent_attr_output['custom_score_overall'] = (similarity * reliability_subtotal).round(3)
# print("Overall score calculation completed.")

# 1. Convert numbers to strings (once for the entire column)
score_overall = dwcagent_attr_output["custom_score_overall"].round(2).astype(str)
score_life = dwcagent_attr_output["custom_score_lifetime_data"].round(1).astype(str)
score_multi = dwcagent_attr_output["custom_score_multiple_names"].round(2).astype(str)

# 2. Vectorized merge
dwcagent_attr_output['attributionRemarks'] = (
    dwcagent_attr_output['namematch_distance_annotation'] + "; " +
    score_overall + " (score overall); " +
    dwcagent_attr_output["life_time_periode"] + " (life time); " +
    score_life + " (life time score); " +
    dwcagent_attr_output["custom_score_lifetime_data_annotation"] + " (life time score note); " +
    score_multi + " (score multiple names);"
)

# adjust dwcagent displayOrder also to olerall score
dwcagent_attr_output.sort_values(
    # by=['namematch_distance', 'family', 'given', 'custom_score_overall'], 
    # ascending=[True, True, True, False], 
    by=['canonical_string_collector_parsed', 'custom_score_overall', 'family', 'given'], 
    ascending=[True, False, True, True], 
    inplace=True
)
# use ordered canonical_string_collector_parsed to generate displayOrder
temp_duplicated = dwcagent_attr_output['canonical_string_collector_parsed'].duplicated() 
    # duplicated() keeps the first value False and mark all other duplicats as True, i.e. we can cumulate the Trues, it gives the order index
temp_insert_value=temp_duplicated.groupby(dwcagent_attr_output['canonical_string_collector_parsed']).cumsum() + 1 # display order starts at 1, incrementing
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('canonical_string_collector_parsed') + 1, 'displayOrder', temp_insert_value, allow_duplicates=True)

# test an show example data
if explain_and_show_the_data:
    print("example data: names having year of birth (yob) < enventDate_min")
    display(dwcagent_attr_output.loc[dwcagent_attr_output['yob_is_lt_eventDate_min'] == True].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_distance",
        # 'yob', 'yod',
        "life_time_periode", 
        'collectors_eventDate_min', 'collectors_eventDate_max', 
        "yob_is_lt_eventDate_min" ,'yod_is_gt_eventDate_max', 
        'custom_score_lifetime_data', 'custom_score_lifetime_data_annotation'
    ]).head(5))
    print("example data: names having year of birth (yob) > eventDate_min")
    display(dwcagent_attr_output.loc[dwcagent_attr_output['yob_is_lt_eventDate_min'] == False].get([
        # "occurrenceID_collectors_firstsample",
        "canonical_string_collector_parsed",
        'itemLabel',
        "custom_score_overall",
        "attributionRemarks",
        'custom_score_multiple_names',
        "namematch_distance",
        # 'yob', 'yod',
        "life_time_periode", 
        'collectors_eventDate_min', 'collectors_eventDate_max', 
        "yob_is_lt_eventDate_min" ,'yod_is_gt_eventDate_max', 
        'custom_score_lifetime_data', 'custom_score_lifetime_data_annotation'
    ]).head(5))


example data: names having year of birth (yob) < enventDate_min


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_distance,life_time_periode,collectors_eventDate_min,collectors_eventDate_max,yob_is_lt_eventDate_min,yod_is_gt_eventDate_max,custom_score_lifetime_data,custom_score_lifetime_data_annotation
0,A,Anna Atkins,0.166,0.2696 (k-means distance); 0.17 (score overall...,0.0,0.2696,1799-1871,1899-08-07,1999-12-10,True,False,0.5,"OK yob + 10, but yod not matching, check name ..."
1,A'buino'o,Albert Frederik Hendrik Buining,0.178,0.5149 (k-means distance); 0.18 (score overall...,0.0,0.5149,1901-1976,1965-08-24,1965-08-24,True,True,1.0,full match
2,A. A. von von Hayek,Alexander von Nordmann,0.133,0.3743 (k-means distance); 0.13 (score overall...,0.0,0.3743,1803-1866,1911-08-01,1911-08-01,True,False,0.5,"OK yob + 10, but yod not matching, check name ..."
4,A. Aalders,Lewis Eldon Aalders,0.465,0.0565 (k-means distance); 0.46 (score overall...,0.0,0.0565,1933-2005,1975-06-20,1975-06-20,True,True,1.0,full match
5,A. Aaronsohn,Aaron Aaronsohn,0.500,0.0 (k-means distance); 0.5 (score overall); 1...,0.0,0.0000,1876-1919,1906-12-06,1907-03-19,True,True,1.0,full match


example data: names having year of birth (yob) > eventDate_min


,canonical_string_collector_parsed,itemLabel,custom_score_overall,attributionRemarks,custom_score_multiple_names,namematch_distance,life_time_periode,collectors_eventDate_min,collectors_eventDate_max,yob_is_lt_eventDate_min,yod_is_gt_eventDate_max,custom_score_lifetime_data,custom_score_lifetime_data_annotation
12,A. Abdullah,Samir K. Abdullah,-0.236,0.0438 (k-means distance); -0.24 (score overal...,0.0,0.0438,1947-?,1957-07-21,1996-05-22,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."
29,A. Adole,Nina Avgustinovna Adolf,-0.287,0.3401 (k-means distance); -0.29 (score overal...,0.0,0.3401,1903-1951,1904-09-01,1969-11-01,False,False,-1.0,life time not matching any eventDate (yob + 10...
43,A. Akkermann,Reiner W.A.P. Akkermans,-0.192,0.1848 (k-means distance); -0.19 (score overal...,0.0,0.1848,1953-?,1961-06-29,1961-06-29,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."
44,A. Akkermans,Reiner W.A.P. Akkermans,-0.236,0.0448 (k-means distance); -0.24 (score overal...,0.0,0.0448,1953-?,1957-06-23,1962-08-04,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."
62,A. Alvarez,Alba Luz Arbeláez Alvarez,-0.250,0.0 (k-means distance); -0.25 (score overall);...,0.0,0.0000,1965-?,1965-12-05,2009-03-27,False,<NA>,-0.5,"yob + 10 not matching, yod unknown, check name..."


In [25]:
column_map_dwcagent_attr = {
    'occurrenceID_collectors_firstsample':'occurrenceID',
    'canonical_string_collector_parsed':  'alternateName',
    'source_data':                        'verbatimName',
    'itemLabel':                          'name',
    'item':                               'identifier',
    'collectors_eventDate_min':           'startedAtTime',
    'collectors_eventDate_max':           'endedAtTime',
    'namematch_distance':                 'custom_namematch_distance'
}
dwcagent_attr_output.rename(
    mapper=column_map_dwcagent_attr,
    axis='columns',
    inplace=True)

dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('identifier')          + 1, 'agentIdentifierType', 'wikidata' , allow_duplicates=True)
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('agentIdentifierType') + 1, 'agentType'          , 'Person'   , allow_duplicates=True)
dwcagent_attr_output.insert(dwcagent_attr_output.columns.get_loc('identifier')          + 1, 'action'             , 'collected', allow_duplicates=True)

if explain_and_show_the_data:
    print("the mapped DarwinCore attribution output examples, sorted by alternateName (=collector name) + displayOrder …")
    display(dwcagent_attr_output.head(20))

dwcagent_attr_output=dwcagent_attr_output.reindex(
    columns=[
        'occurrenceID', # no DwC agent standard (yet)?
        'verbatimName',
        'alternateName',
        'displayOrder', # shall start from 1, 2, 3 …
        'name',
        'attributionRemarks',
        'startedAtTime',
        'endedAtTime',
        'agentType',
        'action',
        'agentIdentifierType',
        'identifier',
        "custom_score_overall", # keep it for calculation convenience, no standard in DwC agent
        'custom_namematch_distance',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_multiple_names',# keep it for calculation convenience, no standard in DwC agent
        'custom_score_lifetime_data' # keep it for calculation convenience, no standard in DwC agent
    ]
)
# column deletion not neccessary after ….reindex(columns=[…])
# for this_column in ['yob', 'yod', 'life_time_periode', 'yob_is_lt_eventDate_min', 'yod_is_gt_eventDate_max', 'score_lifetime_annotation']:
#     del dwcagent_attr_output[this_column]

the mapped DarwinCore attribution output examples, sorted by alternateName (=collector name) + displayOrder …


,occurrenceID,alternateName,displayOrder,family,given,custom_namematch_distance,verbatimName,name,identifier,action,...,yod,namematch_distance_annotation,life_time_periode,yob_is_lt_eventDate_min,yod_is_gt_eventDate_max,custom_score_lifetime_data,custom_score_lifetime_data_annotation,custom_score_multiple_names,custom_score_overall,attributionRemarks
0,https://data.biodiversitydata.nl/naturalis/spe...,A,1,A,NaN,0.2696,A.,Anna Atkins,http://www.wikidata.org/entity/Q264269,collected,...,1871,0.2696 (k-means distance),1799-1871,True,False,0.5,"OK yob + 10, but yod not matching, check name ...",0.0,0.166,0.2696 (k-means distance); 0.17 (score overall...
1,https://data.biodiversitydata.nl/naturalis/spe...,A'buino'o,1,A'buino'o,NaN,0.5149,A'buino'o; Hunt PF,Albert Frederik Hendrik Buining,http://www.wikidata.org/entity/Q241018,collected,...,1976,0.5149 (k-means distance),1901-1976,True,True,1.0,full match,0.0,0.178,0.5149 (k-means distance); 0.18 (score overall...
2,https://data.biodiversitydata.nl/naturalis/spe...,A. A. von von Hayek,1,Hayek,A. A. von,0.3743,Hayek AA von; Hayek F von,Alexander von Nordmann,http://www.wikidata.org/entity/Q466498,collected,...,1866,0.3743 (k-means distance),1803-1866,True,False,0.5,"OK yob + 10, but yod not matching, check name ...",0.0,0.133,0.3743 (k-means distance); 0.13 (score overall...
3,https://data.biodiversitydata.nl/naturalis/spe...,A. Aaku,1,Aaku,A.,0.4229,Aaku A,Kaku,http://www.wikidata.org/entity/Q47119841,collected,...,<NA>,0.4229 (k-means distance),?-?,<NA>,<NA>,0.0,unknown life time,0.0,0.000,0.4229 (k-means distance); 0.0 (score overall)...
4,https://data.biodiversitydata.nl/naturalis/spe...,A. Aalders,1,Aalders,A.,0.0565,Sterk AA; Aalders A,Lewis Eldon Aalders,http://www.wikidata.org/entity/Q21340898,collected,...,2005,0.0565 (k-means distance),1933-2005,True,True,1.0,full match,0.0,0.465,0.0565 (k-means distance); 0.46 (score overall...
5,https://data.biodiversitydata.nl/naturalis/spe...,A. Aaronsohn,1,Aaronsohn,A.,0.0000,Aaronsohn A,Aaron Aaronsohn,http://www.wikidata.org/entity/Q2086130,collected,...,1919,0.0 (k-means distance),1876-1919,True,True,1.0,full match,0.0,0.500,0.0 (k-means distance); 0.5 (score overall); 1...
6,https://data.biodiversitydata.nl/naturalis/spe...,A. Abamo,1,Abamo,A.,0.5450,Abamo A,Jorge A. Abad,http://www.wikidata.org/entity/Q88829543,collected,...,<NA>,0.545 (k-means distance),?-?,<NA>,<NA>,0.0,unknown life time,0.0,0.000,0.545 (k-means distance); 0.0 (score overall);...
7,https://data.biodiversitydata.nl/naturalis/spe...,A. Abang,1,Abang,A.,0.4339,Abbe LB; Abbe EC; Abang A; Bijang H; Jugah ana...,Jorge A. Abad,http://www.wikidata.org/entity/Q88829543,collected,...,<NA>,0.4339 (k-means distance),?-?,<NA>,<NA>,0.0,unknown life time,0.0,0.000,0.4339 (k-means distance); 0.0 (score overall)...
8,https://data.biodiversitydata.nl/naturalis/spe...,A. Abarca,1,Abarca,A.,0.0573,Kappelle M; Abarca A; O,Loudes Abarca,http://www.wikidata.org/entity/Q36610614,collected,...,<NA>,0.0573 (k-means distance),?-?,<NA>,<NA>,0.0,unknown life time,0.0,0.000,0.0573 (k-means distance); 0.0 (score overall)...
9,https://data.biodiversitydata.nl/naturalis/spe...,A. Abassi,1,Abassi,A.,0.0607,Iqbal S; Abassi A,Laleh Abassi Maaf,http://www.wikidata.org/entity/Q21338210,collected,...,<NA>,0.0607 (k-means distance),?-?,<NA>,<NA>,0.0,unknown life time,0.0,0.000,0.0607 (k-means distance); 0.0 (score overall)...


In [26]:
if explain_and_show_the_data:
    print("show column-reduced examples of ?multiple name cases …")
    # criterion = dwcagent_attr_output['alternateName'].map(lambda x: x.startswith('S. Ahmad'))
    criterion = dwcagent_attr_output['custom_score_multiple_names'].map(lambda this_score: this_score < 0 )  # show multiple name cases
    
    # display(dwcagent_attr_output[criterion].head(20))
    display(dwcagent_attr_output[criterion].drop(['agentType', 'action', 'agentIdentifierType'], axis='columns').head(20))

show column-reduced examples of ?multiple name cases …


,occurrenceID,verbatimName,alternateName,displayOrder,name,attributionRemarks,startedAtTime,endedAtTime,identifier,custom_score_overall,custom_namematch_distance,custom_score_multiple_names,custom_score_lifetime_data
10,https://data.biodiversitydata.nl/naturalis/spe...,Abdallah MS-A; Sa'ad FE-ZM; Mahdy M; Abbas A,A. Abbas,1,Alia Abbas,0.0 (k-means distance); -0.25 (score overall);...,1936-02-11,1963-11-01,http://www.wikidata.org/entity/Q60141229,-0.250,0.0000,-0.5,0.0
11,https://data.biodiversitydata.nl/naturalis/spe...,Abdallah MS-A; Sa'ad FE-ZM; Mahdy M; Abbas A,A. Abbas,2,Abdulla Abbas,0.0 (k-means distance); -0.25 (score overall);...,1936-02-11,1963-11-01,http://www.wikidata.org/entity/Q88804360,-0.250,0.0000,-0.5,0.0
74,https://data.biodiversitydata.nl/naturalis/spe...,Fosberg FR; Anderson A,A. Anderson,1,Andrew Anderson,0.0 (k-means distance); 0.0 (score overall); 1...,1934-08-23,1975-07-02,http://www.wikidata.org/entity/Q123652849,0.000,0.0000,-0.5,0.5
75,https://data.biodiversitydata.nl/naturalis/spe...,Fosberg FR; Anderson A,A. Anderson,2,Alexander Anderson,0.0 (k-means distance); 0.0 (score overall); 1...,1934-08-23,1975-07-02,http://www.wikidata.org/entity/Q4718216,0.000,0.0000,-0.5,0.5
77,https://data.biodiversitydata.nl/naturalis/spe...,Andersson A,A. Andersson,1,Karl Alfred Andersson,0.0 (k-means distance); 0.25 (score overall); ...,1887-07-08,1929-08-01,http://www.wikidata.org/entity/Q131724787,0.250,0.0000,-0.5,1.0
76,https://data.biodiversitydata.nl/naturalis/spe...,Andersson A,A. Andersson,2,Axel Andersson,0.0 (k-means distance); -0.5 (score overall); ...,1887-07-08,1929-08-01,http://www.wikidata.org/entity/Q123652899,-0.500,0.0000,-0.5,-0.5
78,https://data.biodiversitydata.nl/naturalis/spe...,Andersson A,A. Andersson,3,I. Anita Andersson,0.0 (k-means distance); -0.5 (score overall); ...,1887-07-08,1929-08-01,http://www.wikidata.org/entity/Q21505194,-0.500,0.0000,-0.5,-0.5
149,https://data.biodiversitydata.nl/naturalis/spe...,Baker A,A. Baker,1,Alan N. Baker,0.0 (k-means distance); 0.0 (score overall); 1...,1902-06-09,1990-06-06,http://www.wikidata.org/entity/Q22105323,0.000,0.0000,-0.5,0.5
148,https://data.biodiversitydata.nl/naturalis/spe...,Baker A,A. Baker,2,Arthur J. D. Baker,0.0 (k-means distance); -0.25 (score overall);...,1902-06-09,1990-06-06,http://www.wikidata.org/entity/Q114860418,-0.250,0.0000,-0.5,0.0
188,https://data.biodiversitydata.nl/naturalis/spe...,Becker A,A. Becker,1,Alexander Becker,0.0 (k-means distance); 0.0 (score overall); 1...,1839-01-01,1990-05-01,http://www.wikidata.org/entity/Q4719276,0.000,0.0000,-0.5,0.5


In [27]:
# TODO further evaluation or filtering, counting, clean up aso.
if not os.path.exists('data'):
    os.makedirs('data')

# this_timestamp_for_data=time.strftime('%Y%m%d') # 20230719
# this_timestamp_for_data=20231116
this_output_file='data/results_naturalis_collectors-eventDate_vs_wikidata-botanists_kneighbor_dwc-agent-output_%s.csv' % (
    this_timestamp_for_data
)

dwcagent_attr_output.to_csv(this_output_file, index=False)

print("Wrote matches of collector names as dwc-agent-output into %s (%d kB)" % 
    (this_output_file, os.path.getsize(this_output_file) >> 10 ) # 10000 >> 10 = bitshift operator, to get kilo bytes (10-bits=>1024)
)

Wrote matches of collector names as dwc-agent-output into data/results_naturalis_collectors-eventDate_vs_wikidata-botanists_kneighbor_dwc-agent-output_20260210.csv (24588 kB)


## Documentation

Explanation of columns:

Column | Description
-|-
**Botanical collectors** |
family | parsed family name
given | parsed given name
suffix | suffix from name parsing
particle | particle from name parsing
dropping_particle | dropping_particle from name parsing
nick | nick name from name parsing
appellation | appellation from name parsing
title | title from name parsing
eventDate | date of the sampling event (required by GBIF, ☞ https://www.gbif.org/data-quality-requirements-sampling-events)
eventDate_min | calculated earliest date of all the sampling events within the data
eventDate_max | calculated latest date of all the sampling events within the data
eventDate_mean | calculated mean date of all the sampling events within the data
TODO activity_span | Number of years between first and last collection
**Name matching** |
nammatch_collector | matched name of the data set
nammatch_wikidata | matched name; = Wikidata item label name is matched to
namematch_distance | Nearest Neighbour distance between the name and matched name; the lower the value, the better the match
**DarwinCore Agent Output** | (☞ [agent_actions_v2020-09-08.xml](https://github.com/tdwg/attribution/blob/master/people/dwc/agent_actions_v2020-09-08.xml))
occurrenceID | occurrence ID of the data item
name | the interpreted name match (https://github.com/tdwg/attribution/ The name of the item. In this case the *full name* as would be written on a legal document (without abbreviation), eg givenName familyName)
verbatimName | the source data name(s) (https://github.com/tdwg/attribution/ As written on occurrence, such as the collection or determination label.)
alternateName | the input name, collector source name (An alias for the item. Other full name agent may have been known under such as maiden name.)
displayOrder | I guess ordering the multiple name cases (https://github.com/tdwg/attribution/ The display order for the agent that executed the action when more than one agent was a participant.)
attributionRemarks | notes on the results (distance or similarity), including calculated value
agentType | The nature of the agent, e.g. "Person", "Organization", "SoftwareApplication"
action | The name of the single action written as a verb in past tense. Recommended best practice is to use a controlled vocabulary, examples "collected" or "identified"
agentIdentifierType | The type of identifier for the agent. (https://github.com/tdwg/attribution/ Recommended best practice is to use a controlled vocabulary, e.g. “ORCID”, “ISNI”, “Wikidata”, “VIAF”, “RoR”, “Ringgold”, “GRID”).
identifier | Wikidata ID (Recommended practice is to identify the resource by means of a string conforming to an identification system. Examples include International Standard Book Number (ISBN), Digital Object Identifier (DOI), and Uniform Resource Name (URN). Persistent identifiers should be provided as HTTP URIs.)
startedAtTime | (https://github.com/tdwg/attribution/ Start is when an action is deemed to have been started by an agent.) the first date of eventDate (supposedly the first sampling date), but grouped from collector name—in case of multiple name matches this first “sampling date” is less reliable and be reliable to relate to the source collector’s life time.
endedAtTime | (https://github.com/tdwg/attribution/ End is when an action is deemed to have been ended by an agent.) the last date of eventDate (supposedly the last sampling date), but grouped from collector name—in case of multiple name matches this first “sampling date” is less reliable and be reliable to relate to the source collector’s life time.
**Wikidata** |
item | Wikidata Item ID (URL)
itemLabel | Wikidata Item label
surname	| Surname; derived from item label
initials | Initials; derived from item label
canonical_string | Canonical name string; derived from item label, used for matching
orcid | ORCID ([P496](https://www.wikidata.org/wiki/Property:P496))
viaf | VIAF ID ([P214](https://www.wikidata.org/wiki/Property:P214))
isni | ISNI ID ([P213](https://www.wikidata.org/wiki/Property:P496))	
harv | Harvard Index of Botanists ID ([P6264](https://www.wikidata.org/wiki/Property:P6264))
ipni | IPNI author ID ([P586](https://www.wikidata.org/wiki/Property:P586))
abbr | botanist author abbreviation (standard form) ([P428](https://www.wikidata.org/wiki/Property:P428))
bionomia_id | identifier for a collector and/or determiner of natural history specimens, in the Bionomia database ([P6944](https://www.wikidata.org/wiki/Property:P6944))
yob	| Year of birth (derived from [P569](https://www.wikidata.org/wiki/Property:P569))
yod	| Year of death (derived from [P496](https://www.wikidata.org/wiki/Property:P570))
wyb	| Work year period begin ([P2031](https://www.wikidata.org/wiki/Property:P2031))
wye | Work year period end ([P2032](https://www.wikidata.org/wiki/Property:P2032))

Refactoring from <https://github.com/nielsklazenga/avh-collectors/blob/master/match_names_to_wikidata_items.ipynb>

AVH | collector_matching (here)
-|-
avh_matches | collectors_all_matches
wd_test | wd_matchtest